In [1]:
import pickle
data=None
with open('data_intervals.pkl', 'rb') as f:
    data = pickle.load(f)

In [10]:
data['A-10030']['twoweeks'].shape

(4032,)

In [11]:
combined_data = np.concatenate((data['A-10030']['twoweeks'], data['A-10030']['day']))

# Split the combined array into groups of 288 values
num_days = combined_data.shape[0] // 288
daily_data = combined_data.reshape((num_days, 288))


In [13]:
daily_data.shape

(15, 288)

In [14]:
import numpy as np
import pywt

def detect_disruption(observations, threshold=1.5):
    # Select the last day of observations
    last_day_obs = observations[-1]
    
    # Apply Daubechies wavelet decomposition
    coeffs = pywt.wavedec(last_day_obs, wavelet='db4', level=5)
    
    # Get the detail coefficients from the decomposition
    details = coeffs[1:]

    # Calculate the standard deviation of detail coefficients
    std_devs = [np.std(detail) for detail in details]

    # Find the average standard deviation
    avg_std_dev = np.mean(std_devs)

    # Detect disruption by comparing each detail's standard deviation to the average
    disruptions = [i for i, std_dev in enumerate(std_devs) if std_dev > threshold * avg_std_dev]

    return disruptions

# Example usage
observations = daily_data#np.random.rand(14, 288)
disruptions = detect_disruption(observations)

print("Disruptions detected at levels:", disruptions)

Disruptions detected at levels: [0, 1]


In [16]:
import numpy as np
import pywt

def detect_disruption(observations, threshold=1.5):
    # Select the last day of observations
    last_day_obs = observations[-1]
    
    # Apply Daubechies wavelet decomposition
    coeffs = pywt.wavedec(last_day_obs, wavelet='db4', level=5)
    
    # Get the detail coefficients from the decomposition
    details = coeffs[1:]

    # Calculate the standard deviation of detail coefficients
    std_devs = [np.std(detail) for detail in details]

    # Find the average standard deviation
    avg_std_dev = np.mean(std_devs)

    # Detect disruption by comparing each detail's standard deviation to the average
    disruption_levels = [i for i, std_dev in enumerate(std_devs) if std_dev > threshold * avg_std_dev]

    # Calculate exact intervals of disruption
    disruption_intervals = []
    for level in disruption_levels:
        detail = details[level]
        disruption_indices = np.where(np.abs(detail) > threshold * avg_std_dev)[0]
        
        for idx in disruption_indices:
            # Set all other details to zero
            modified_coeffs = [coeffs[0]] + [(detail if i == level else np.zeros_like(detail)) for i, detail in enumerate(details)]
            
            # Reconstruct the original signal using the modified coefficients
            reconstructed_signal = pywt.waverec(modified_coeffs, 'db4')
            
            disruption_intervals.append((reconstructed_signal[idx], reconstructed_signal[idx+1]))
    
    return disruption_intervals

# Example usage
observations = daily_data
disruption_intervals = detect_disruption(observations)

print("Disruptions observed at intervals:", disruption_intervals)


Disruptions observed at intervals: [(65.09694836278669, 64.88107642760461), (64.61980618786028, 64.55474975249832), (64.55474975249832, 64.49900257368877), (64.49900257368877, 64.45005730189463), (64.45005730189463, 64.45068286671143), (64.49305720847254, 64.51017122338355), (64.51017122338355, 64.50255780224843), (63.51752439647645, 63.64375852788479), (63.64375852788479, 63.874721151862246)]
